# ROS Parameter Server

- The ROS Parameter Server can store strings, integers, floats, booleans, lists, dictionaries, iso8601 dates, and base64-encoded data. Dictionaries must have string keys.

# 1) Lista dostępnych parametrów

In [1]:
import rospy
rospy.get_param_names()

['/run_id',
 '/roslaunch/uris/host_localhost__43645',
 '/rosversion',
 '/rosdistro',
 '/turtlesim/background_r',
 '/turtlesim/background_g',
 '/turtlesim/background_b']

# 2) Odczyt wartości parametru

In [3]:
print(rospy.get_param("turtlesim/background_r"))
print(rospy.get_param("turtlesim/background_g"))
print(rospy.get_param("turtlesim/background_b"))

69
86
255


# 3) Ustawienie nowych wartości parametrów tła

In [4]:
rospy.set_param("turtlesim/background_r", 255)
rospy.set_param("turtlesim/background_g", 0)
rospy.set_param("turtlesim/background_b", 0)

### Uwaga:
- Aby wprowadzone zmiany były widoczne należy wywołać dodatkowo serwis */clear*

In [5]:
!rosservice call /clear

# 4) Ustawianie kilku różnych parametrów w określonej przestrzeni nazw.

- Na przykładzie kolorów RGB tła można by ustawić je w przestrzeni nazw **color** i zgrupować razem.
- Takie ustawienie parametrów nie będzie miało wpływu na środowisko symulacyjne, gdyż nie są one obsługiwane.

### Wersja 1

In [6]:
grupa = "color"
rospy.set_param(grupa+"/r", 41)
rospy.set_param(grupa+"/g", 231)
rospy.set_param(grupa+"/b", 0)

In [7]:
rospy.get_param_names()

['/run_id',
 '/roslaunch/uris/host_localhost__46703',
 '/rosversion',
 '/rosdistro',
 '/turtlesim/background_r',
 '/turtlesim/background_g',
 '/turtlesim/background_b',
 '/color/r',
 '/color/g',
 '/color/b']

- Umożliwia nam wyświetlenie parametrów z określonej grupy. 
- Na wyjściu get_param otrzymujemy słownik z wartościami parametrów w przestrzeni nazw:

In [8]:
rospy.get_param("color")

{'r': 41, 'g': 231, 'b': 0}

### Wersja 2 - Parametry przekazywane jako słownik

In [9]:
rospy.set_param("color2", {"r": 133, "g":43, "b":25})
rospy.get_param("color2")

{'r': 133, 'g': 43, 'b': 25}

In [10]:
rospy.get_param_names()

['/run_id',
 '/roslaunch/uris/host_localhost__46703',
 '/rosversion',
 '/rosdistro',
 '/turtlesim/background_r',
 '/turtlesim/background_g',
 '/turtlesim/background_b',
 '/color/r',
 '/color/g',
 '/color/b',
 '/color2/r',
 '/color2/g',
 '/color2/b']

# 5) Usuwanie parametrów 
- Usunięcie pojedynczego parametru:

In [ ]:
rospy.delete_param("/color2/r")
rospy.get_param_names()

- Usunięcie wszystkich parametrów:

In [ ]:
rospy.delete_param("/color2/")
rospy.get_param_names()

# 6) Zadania

#### Zadanie 1
Napisać funkcję, która będzie generwała parametry dla robota o podanej nazwie. Parametry powinny być tak wygenerowane, aby znajdowały się w przestrzeni nazw robota o podanej w argumencie nazwie. Parametry pojawiające się w tej przestrzeni powinny odpowiadać: maksymalnej dopuszczalnej prędkości postępowej, obrotowej, trybu pracy (auto/manual). Można dopisać jakieś własne parametry.

In [23]:
def set_params(name):
    rospy.set_param(name+"/maks_predkosc_postepowa", 10)
    rospy.set_param(name+"/maks_predkosc_obrotowa",5)
    rospy.set_param(name+"/tryb_pracy","auto")
    return rospy.get_param(name)

In [24]:
set_params('turtle1')

{'maks_predkosc_postepowa': 10,
 'maks_predkosc_obrotowa': 5,
 'tryb_pracy': 'auto'}

In [17]:
try:
    wazny_parametr = rospy.get_param("fff")
except KeyError:
    wazny_parametr = "domyslna_wartosc"

#### Zadanie 2
1. Skopiować klasę Robot z zajęć o serwisach wraz z importem odpowiednich bibliotek. Rozbudować ją o napisaną w poprzednim zadaniu funkcję. Podczas inicjalizacji klasy Robot powinny się również pojawić parametry robota.  Funkcja jako metoda klasy powinna mieć jako pierwszy argument self wskazujący na klasę.

2. Dopisać metodę do wyświetlania parametrów robota z jego przestrzeni nazw.
3. Uzupełnić metodę **unregistered** o usuwanie wszystkich parametrów robota.
4. Dodać do klasy serwis od usuwania robota. (Serwis /kill)
5. Utworzyć metodę do wyświetlania listy parametrów robota. Odczytać parametry z przestrzeni nazw i wyświetlić.

In [2]:
import rospy
from turtlesim.srv import Spawn
from turtlesim.msg import Pose
from geometry_msgs.msg import Twist
from pkg_tsr.srv import Task
import math
from turtlesim.srv import Kill

rospy.init_node("serwis_node_test")

class Robot:
    def __init__(self, name, x, y):
        self.name = name
        self.pose = Pose()
        create_new_turtle = rospy.ServiceProxy('spawn', Spawn)
        create_new_turtle(x, y, 0, name)
        
        self.velocity_pub = rospy.Publisher(name+'/cmd_vel', Twist, queue_size=10)
        self.pose_sub = rospy.Subscriber(name+'/pose',Pose, self.callback_pose)
        self.follow_service = rospy.Service(name+'/go_to_goal', Task, self.go_to_goal)
        
        rospy.set_param(self.name+"/maks_predkosc_postepowa", 10)
        rospy.set_param(self.name+"/maks_predkosc_obrotowa",5)
        rospy.set_param(self.name+"/tryb_pracy","auto")
        
        self.kill_service = rospy.ServiceProxy('kill',Kill)
        
    def get_params(self):
        try:
            return rospy.get_param(self.name)
        except KeyError:
            print("Brak parametrow")
    
    def go_to_goal(self, msg_srv):
        max_speed = 0.3
        omega=0.2
        tolerance = 0.05
        
        #polozenie celu
        x_c=msg_srv.x
        y_c=msg_srv.y
        
        msg_vel=Twist()
        
        while True:
            #polozenie robota
            x=self.pose.x
            y=self.pose.y
            theta=self.pose.theta
            
            msg_vel.linear.x=max_speed
            if math.atan2(y_c-y,x_c-x)!=theta:
                if theta<math.atan2(y_c-y,x_c-x):
                    msg_vel.angular.z=omega
                else:
                    msg_vel.angular.z=-omega
            else:
                msg_vel.angular.z=0
            if math.sqrt((x_c-x)**2+(y_c-y)**2)<tolerance: 
                return "finished"
            self.velocity_pub.publish(msg_vel)
    
    def callback_pose(self, pose_msg):
        self.pose=pose_msg
    def unregister(self):
        self.follow_service.shutdown()
        rospy.delete_param("/"+self.name+"/")
    
    def kill(self):
        self.kill_service(self.name)
        self.unregister()

In [3]:
!rosservice call reset

In [75]:
new_robot.unregister()

In [4]:
new_robot=Robot('Delmiak',4,5)

In [7]:
new_robot.get_params()

Brak parametrow
shutdown request: [/serwis_node_test] Reason: new node registered with same name


In [6]:
new_robot.kill()